In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets.mnist import MNIST
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import optim
import time
from tqdm import tqdm
import os
from torch.utils.data import Dataset
from PIL import Image

In [2]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 

In [6]:
train_path = '/home/ielab/dataset/ms1m_dataset/train_gan/'
test_path = '/home/ielab/dataset/ms1m_dataset/test_gan'

og_test_path = '/home/ielab/dataset/ms1m_dataset/test/'

In [7]:
col_list = os.listdir(train_path)

lb = {string : i for i,string in enumerate(col_list)}

In [8]:
col_list

['1977',
 '2782',
 '1236',
 '2546',
 '1867',
 '1647',
 '1458',
 '1686',
 '2647',
 '1504',
 '1148',
 '2373',
 '2246',
 '3262',
 '407',
 '1569',
 '226',
 '1663',
 '3590',
 '172',
 '1991',
 '192',
 '3183',
 '183',
 '1170',
 '1191',
 '256',
 '1618',
 '1731',
 '2076',
 '203',
 '3264',
 '1791',
 '2709',
 '2362',
 '2053',
 '2657',
 '3002',
 '1525',
 '2954',
 '2820',
 '2903',
 '3372',
 '2719',
 '1141',
 '2222',
 '2730',
 '265',
 '348',
 '1944',
 '3036',
 '1443',
 '1039',
 '3522',
 '1414',
 '1425',
 '1473',
 '1697',
 '2724',
 '1902',
 '1355',
 '2385',
 '466',
 '2276',
 '1015',
 '3338',
 '1481',
 '1433',
 '2144',
 '3578',
 '2529',
 '1486',
 '2157',
 '3501',
 '1259',
 '3095',
 '2240',
 '2883',
 '1915',
 '1004',
 '1107',
 '472',
 '2467',
 '1490',
 '2391',
 '148',
 '3345',
 '2387',
 '413',
 '2111',
 '2326',
 '2694',
 '2840',
 '1285',
 '2279',
 '1658',
 '3350',
 '3524',
 '1442',
 '1570',
 '2428',
 '2079',
 '2088',
 '1153',
 '2571',
 '3541',
 '1718',
 '2159',
 '2539',
 '2744',
 '2244',
 '1319',
 '336

In [9]:
lb

{'1977': 0,
 '2782': 1,
 '1236': 2,
 '2546': 3,
 '1867': 4,
 '1647': 5,
 '1458': 6,
 '1686': 7,
 '2647': 8,
 '1504': 9,
 '1148': 10,
 '2373': 11,
 '2246': 12,
 '3262': 13,
 '407': 14,
 '1569': 15,
 '226': 16,
 '1663': 17,
 '3590': 18,
 '172': 19,
 '1991': 20,
 '192': 21,
 '3183': 22,
 '183': 23,
 '1170': 24,
 '1191': 25,
 '256': 26,
 '1618': 27,
 '1731': 28,
 '2076': 29,
 '203': 30,
 '3264': 31,
 '1791': 32,
 '2709': 33,
 '2362': 34,
 '2053': 35,
 '2657': 36,
 '3002': 37,
 '1525': 38,
 '2954': 39,
 '2820': 40,
 '2903': 41,
 '3372': 42,
 '2719': 43,
 '1141': 44,
 '2222': 45,
 '2730': 46,
 '265': 47,
 '348': 48,
 '1944': 49,
 '3036': 50,
 '1443': 51,
 '1039': 52,
 '3522': 53,
 '1414': 54,
 '1425': 55,
 '1473': 56,
 '1697': 57,
 '2724': 58,
 '1902': 59,
 '1355': 60,
 '2385': 61,
 '466': 62,
 '2276': 63,
 '1015': 64,
 '3338': 65,
 '1481': 66,
 '1433': 67,
 '2144': 68,
 '3578': 69,
 '2529': 70,
 '1486': 71,
 '2157': 72,
 '3501': 73,
 '1259': 74,
 '3095': 75,
 '2240': 76,
 '2883': 77,
 '1915

In [10]:
batch_size = 64

In [11]:
num_classe = len(col_list)

In [12]:
trans = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    ])

In [13]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.dir_list = os.listdir(img_dir)
        self.transform = transform
        self.target_transform = target_transform

        self.files = []
        for folder in self.dir_list:
          folder_list = os.listdir(os.path.join(img_dir, folder))
          for file in folder_list:
            self.files.append([
                os.path.join(self.img_dir, folder+'/'+file),
                folder])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = Image.open(self.files[idx][0])
        image = image.convert('RGB')
        image = self.transform(image)

        label = self.files[idx][1]
        label = lb[label]

        return image, label

In [14]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

trainset = CustomDataset(train_path, trans)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = CustomDataset(test_path, trans)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

In [15]:
class ArcFaceLoss(nn.Module):
    def __init__(self, num_classes, embedding_size, margin, scale):
        """
        ArcFace: Additive Angular Margin Loss for Deep Face Recognition
        (https://arxiv.org/pdf/1801.07698.pdf)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
            margin: m in the paper, the angular margin penalty in radians
            scale: s in the paper, feature scale
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.margin = margin
        self.scale = scale
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        cosine = self.get_cosine(embeddings) # (None, n_classes)
        mask = self.get_target_mask(labels) # (None, n_classes)
        cosine_of_target_classes = cosine[mask == 1] # (None, )
        modified_cosine_of_target_classes = self.modify_cosine_of_target_classes(
            cosine_of_target_classes
        ) # (None, )
        diff = (modified_cosine_of_target_classes - cosine_of_target_classes).unsqueeze(1) # (None,1)
        logits = cosine + (mask * diff) # (None, n_classes)
        logits = self.scale_logits(logits) # (None, n_classes)
        return logits
        
    def get_cosine(self, embeddings):
        """
        Args:
            embeddings: (None, embedding_size)
        Returns:
            cosine: (None, n_classes)
        """
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.W))
        return cosine
    
    def get_target_mask(self, labels):
        """
        Args:
            labels: (None,)
        Returns:
            mask: (None, n_classes)
        """
        batch_size = labels.size(0)
        onehot = torch.zeros(batch_size, self.num_classes, device=labels.device)
        onehot.scatter_(1, labels.unsqueeze(-1), 1)
        return onehot
        
    def modify_cosine_of_target_classes(self, cosine_of_target_classes):
        """
        Args:
            cosine_of_target_classes: (None,)
        Returns:
            modified_cosine_of_target_classes: (None,)
        """
        eps = 1e-6
        # theta in the paper
        angles = torch.acos(torch.clamp(cosine_of_target_classes, -1 + eps, 1 - eps))
        return torch.cos(angles + self.margin)
    
    def scale_logits(self, logits):
        """
        Args:
            logits: (None, n_classes)
        Returns:
            scaled_logits: (None, n_classes)
        """
        return logits * self.scale
    
class SoftmaxLoss(nn.Module):
    def __init__(self, num_classes, embedding_size):
        """
        Regular softmax loss (1 fc layer without bias + CrossEntropyLoss)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        logits = F.linear(embeddings, self.W)
        return nn.CrossEntropyLoss()(logits, labels)

In [16]:
class Embedder(nn.Module):
    def __init__(self, embedding_size):
        super().__init__()
        self.model = torchvision.models.resnet50(pretrained = False)
        self.dropout = nn.Dropout(0.5)
        
        self.classifier = nn.Linear(1000, embedding_size)

    def forward(self, images):
        outputs = self.model(images)
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        return outputs

In [17]:
# Epoch 반복 횟수
repeat_num = 3


embedding_size = 512
max_epochs = 100

embedder = Embedder(embedding_size=embedding_size)
arcface = ArcFaceLoss(num_classes=num_classe, embedding_size=embedding_size,margin=0.3, scale=30.0)

if torch.cuda.device_count() > 1:
    embedder = nn.DataParallel(embedder)
    arcface = nn.DataParallel(arcface)
embedder = embedder.to(device)
arcface = arcface.to(device)

optimizer = optim.Adam(embedder.parameters(), lr=1e-3 ) 
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.9)


for num in range(repeat_num):
    start = time.time()
    print(f'epoch :{num}')
    for epoch in range(max_epochs):
        e_time = time.time()
        for i, (images, labels) in enumerate(trainloader):
            embedder.train()
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            embeddings = embedder(images)

            logits = arcface(embeddings, labels)
            loss = criterion(logits, labels)
            loss.backward() 
            optimizer.step() 
        
        print(f'Epoch: {epoch} - Batch: {i*batch_size} - Loss: {loss:.6f} - Time:{time.time() - e_time}')
        with open('/home/ielab/project/samples/(test)gan_loss-'+str(num) +'.txt', 'a') as file:
            file.write(f'{loss:.6f}\n')
                    
    torch.save(embedder, './' + str(num)  + '(test)gan_model.pth')
    
    #  정확도 측정
    train_results = []
    train_labels = []
    test_results = []
    test_labels = []

    embedder.eval()
    with torch.no_grad():
      for img, label in trainloader:
        img = img.cuda()
        train_results.append(embedder(img).cpu().detach().numpy())
        train_labels.append(label)

    train_results = np.concatenate(train_results)
    train_labels = np.concatenate(train_labels)

    embedder.eval()
    with torch.no_grad():
      for img, label in testloader:
        img = img.cuda()
        test_results.append(embedder(img).cpu().detach().numpy())
        test_labels.append(label)

    test_results = np.concatenate(test_results)
    test_labels = np.concatenate(test_labels)
    
    # kNN 모델 선언
    k = 50
    model = KNeighborsClassifier(n_neighbors = k)
    # 모델 학습
    model.fit(train_results, train_labels)
    #knn 검증
    train_pred = model.predict(train_results)
    train_acc = (train_pred == train_labels).mean()
    with open('/home/ielab/project/samples/(test)gan_train_acc-'+str(num) +'.txt', 'a') as file:
        file.write(f'{train_acc:.6f}\n')
    print(f'train acc : {train_acc}')
    test_pred = model.predict(test_results)
    test_acc = (test_pred == test_labels).mean()
    with open('/home/ielab/project/samples/(test)gan_test_acc-'+str(num) +'.txt', 'a') as file:
        file.write(f'{test_acc:.6f}\n')
    print(f'test acc : {test_acc}')
    # F1-Score
    f1 = metrics.classification_report(test_labels, test_pred)
    with open('/home/ielab/project/samples/(test)gan_test_f1-'+str(num) +'.txt', 'a') as file:
        file.write(f'{f}')

/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


epoch :0


KeyboardInterrupt: 

In [ ]:
embedder = torch.load("0gan_model.pth", map_location=device)
print(embedder)

In [ ]:
#  정확도 측정
train_results = []
train_labels = []
test_results = []
test_labels = []

embedder.eval()
with torch.no_grad():
  for img, label in trainloader:
    img = img.cuda()
    train_results.append(embedder(img).cpu().detach().numpy())
    train_labels.append(label)

train_results = np.concatenate(train_results)
train_labels = np.concatenate(train_labels)

embedder.eval()
with torch.no_grad():
  for img, label in testloader:
    img = img.cuda()
    test_results.append(embedder(img).cpu().detach().numpy())
    test_labels.append(label)

test_results = np.concatenate(test_results)
test_labels = np.concatenate(test_labels)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


# kNN 모델 선언
k = 50
model = KNeighborsClassifier(n_neighbors = k)
# 모델 학습
model.fit(train_results, train_labels)
#knn 검증
train_pred = model.predict(train_results)
train_acc = (train_pred == train_labels).mean()
with open('/home/ielab/project/samples/gan_train_acc.txt', 'a') as file:
    file.write(f'{train_acc:.6f}\n')
print(f'train acc : {train_acc}')
test_pred = model.predict(test_results)
test_acc = (test_pred == test_labels).mean()
with open('/home/ielab/project/samples/gan_test_acc.txt', 'a') as file:
    file.write(f'{test_acc:.6f}\n')
print(f'test acc : {test_acc}')

In [ ]:
# F1-Score
f1 = metrics.classification_report(test_labels, test_pred)
with open('/home/ielab/project/samples/gan_test_f1.txt', 'a') as file:
    file.write(f'{f1}\n')

In [ ]:
print(f1)